# Automatic parameter sweeping & result visualizaiton
This notebook demonstrates our ability to sweep over a set of parameters for markets and pricing models. The final results are displayed via a [parallel coordinates plot](https://arxiv.org/abs/1705.00368), which illustrates how each parameter influences a final metric, such as net fees earned.

In [ ]:
import time

import numpy as np
import pandas as pd
import itertools
import plotly.graph_objects as go

from elfpy.simulators import YieldSimulator
from elfpy.utils.data import format_trades

In [ ]:
start_time = time.time()
num_runs = 0

config_file = "../../config/example_config.toml"
simulator = YieldSimulator(config_file)
simulator_rng = np.random.default_rng(simulator.config.simulator.random_seed)
simulator.reset_rng(simulator_rng)
simulator.set_random_variables()

pricing_models = ['Element']#, 'YieldSpacev2']
floor_fees = [0, 5, 10] #bps
vault_ages = [0.1, 0.3, 0.5] # normalized year
init_pool_apys = [0.02, 0.05, 0.20] # decimal percent
vault_apys = [[i,]*simulator.config.simulator.num_trading_days for i in [0.03, 0.05, 0.09]] # multiple constant vault apy

for (
    pricing_model,
    vault_age,
    vault_apy,
    init_pool_apy,
    floor_fee
) in itertools.product(
    pricing_models,
    vault_ages,
    vault_apys,
    init_pool_apys,
    floor_fees
):
    if pricing_model == 'Element' and floor_fee > 0:
        pass
    override_dict = {
        'pricing_model_name': pricing_model,
        'vault_apy': vault_apy,
        'init_vault_age': vault_age,
        'init_pool_apy': init_pool_apy,
        'floor_fee': floor_fee
    }
    simulator.run_simulation(override_dict)
    num_runs += 1

end_time = time.time()
print(f'Total time for {num_runs} runs was {end_time-start_time:.3f} seconds; which is {(end_time-start_time)/num_runs:.3f} seconds per run')

In [ ]:
[trades, df_fees_volume] = format_trades(simulator.analysis_dict)
model_numbers = []
for name in trades['model_name']:
    if name == 'Element':
        model_numbers.append(0)
    elif name == 'YieldSpacev2':
        model_numbers.append(1)
    elif name == 'YieldSpacev2MinFee':
        model_numbers.append(2)
trades['model_number'] = model_numbers

preserved_columns = [
    'day',
    'model_name',
    'model_number',
    'init_vault_age',
    'price_total_return_scaled_to_share_price',
]
trades_reduced = trades.groupby(preserved_columns).agg({
    'trade_volume_usd': ['sum'],
    'fee_in_usd': ['mean', 'sum'],
    'fee_in_bps': ['mean', 'sum'],
    'pool_apy': ['mean'],
    'vault_apy': ['mean'],
})
trades_reduced.columns = ['_'.join(col).strip() for col in trades_reduced.columns.values]
trades_reduced = trades_reduced.reset_index()
#trades_reduced['spot_price_growth'] = (trades_reduced.groupby(['day'])['price_total_return_scaled_to_share_price'].apply(pd.Series.pct_change) + 1)
#trades_reduced['spot_price_growth_sum'] = trades_reduced.groupby(['day']).agg({'spot_price_growth':['sum']})

trades_reduced

In [ ]:
fig = go.Figure(data=
    go.Parcoords(
        line = dict(color = trades_reduced.fee_in_usd_sum,
                   #colorscale = 'Electric',
                   colorscale = 'viridis',
                   #colorscale = 'tealrose',
                   showscale = True,
                   cmin = trades_reduced.fee_in_usd_sum.min(),
                   cmax = trades_reduced.fee_in_usd_sum.max()),
        dimensions = list([
            dict(tickvals = [0, 1, 2],
                 ticktext = ['Element', 'YieldSpacev2', 'YieldSpacev2MinFee'],
                 label = 'Model Type',
                 values = trades_reduced.model_number),
            dict(range = [trades_reduced.init_vault_age.min(), trades_reduced.init_vault_age.max()],
                 #tickvals = list(range(trades_reduced.init_vault_age.min(), trades_reduced.init_vault_age.max()+2, 5)),
                 label = 'Initial Vault Age',
                 values = trades_reduced.init_vault_age),
            dict(range = [trades_reduced.vault_apy_mean.min(), trades_reduced.vault_apy_mean.max()],
                 label = "Vault APY",
                 values = trades_reduced.vault_apy_mean),
            dict(range = [trades_reduced.pool_apy_mean.min(), trades_reduced.pool_apy_mean.max()],
                 label = 'Mean Pool APY',
                 values = trades_reduced.pool_apy_mean),
            dict(range = [trades_reduced.fee_in_usd_sum.min(), trades_reduced.fee_in_usd_sum.max()],
                 label = 'Fee (USD)',
                 values = trades_reduced.fee_in_usd_sum),
        ])
    )
)

fig.show()